In [2]:
import numpy as np
from datetime import datetime
from kadlu.geospatial.data_sources import chs
from kadlu.geospatial.data_sources import era5
from kadlu.geospatial.data_sources import hycom
from kadlu.geospatial.data_sources import wwiii
from kadlu.geospatial.data_sources.fetch_util import database_cfg
from kadlu.geospatial.data_sources.fetch_util import dt_2_epoch

#from kadlu.geospatial.data_sources import fetch_util

conn, db = database_cfg()

In [3]:
from importlib import reload
reload(wwiii)
#reload(chs)

<module 'kadlu.geospatial.data_sources.wwiii' from '/home/matt_s/kadlu/kadlu/geospatial/data_sources/wwiii.py'>

## Parameter Selection 

In [4]:
temp_salinity_source = hycom.Hycom()
bathysource = chs.Chs()
wavesource1 = era5.Era5()
wavesource2 = wwiii.Wwiii()

In [5]:
# parameter selection
start = datetime(2015, 3, 9, 13, 25, 0)
end   = datetime(2015, 3, 9, 19, 0, 0)
top, bottom = 0, 0 

# gulf st lawrence
south, west = 44.013, -70.429
north, east = 51.823, -55.977

# hudsons bay
#south, west = 51.388, -95.321
#north, east = 64.068, -75.703
# hudsons bay subset
#north, east = 55.039, -78.740

## Fetching

In [16]:
temp_salinity_source.fetch_salinity(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

temp_salinity_source.fetch_temp(south=south, north=north, west=west, east=east, 
                      start=start, end=end, top=top, bottom=bottom)

downloading 107016 salinity values from hycom...
downloaded 102.4Kb in 2.472s. parsed and inserted 0 rows in 0.403s
60672 null values removed, 46344 duplicate rows ignored

downloading 107016 water_temp values from hycom...
downloaded 108.8Kb in 1.726s. parsed and inserted 0 rows in 0.400s
60672 null values removed, 46344 duplicate rows ignored



In [ ]:
bathysource.fetch_bathymetry(south=south, north=north, west=west, east=east)

In [4]:
wavesource1.fetch_windwaveswellheight(start=start, end=end)

2019-12-18 15:30:21,338 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


downloading ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_13h.grb2 from Copernicus Climate Data Store...


2019-12-18 15:30:22,021 INFO Request is completed
2019-12-18 15:30:22,022 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data0/adaptor.mars.internal-1576697374.0180411-7507-19-e7eb0125-19cb-49f8-baf2-3a9867aa1c33.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_13h.grb2 (312.1K)
2019-12-18 15:30:22,932 INFO Download rate 343K/s  
2019-12-18 15:30:26,126 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


processed and inserted 143420 rows. 116500 null values removed, 0 duplicate rows ignored

downloading ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_14h.grb2 from Copernicus Climate Data Store...


2019-12-18 15:30:26,873 INFO Request is queued
2019-12-18 15:30:28,029 INFO Request is completed
2019-12-18 15:30:28,030 INFO Downloading http://136.156.132.110/cache-compute-0001/cache/data9/adaptor.mars.internal-1576697426.9646833-4773-3-fa182250-4fd8-40bc-8459-73465ec39619.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_14h.grb2 (312.1K)
2019-12-18 15:30:28,932 INFO Download rate 346.3K/s
2019-12-18 15:30:32,008 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


...

processed and inserted 143420 rows. 116500 null values removed, 0 duplicate rows ignored

downloading ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_18h.grb2 from Copernicus Climate Data Store...


2019-12-18 15:30:52,896 INFO Request is queued
2019-12-18 15:30:54,054 INFO Request is completed
2019-12-18 15:30:54,055 INFO Downloading http://136.156.133.36/cache-compute-0010/cache/data8/adaptor.mars.internal-1576697453.3805776-13980-31-05a4fff1-3190-40cf-b963-6fa0a9e9780d.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_18h.grb2 (312.1K)
2019-12-18 15:30:56,399 INFO Download rate 133.2K/s
2019-12-18 15:30:59,764 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


processed and inserted 143420 rows. 116500 null values removed, 0 duplicate rows ignored

downloading ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_19h.grb2 from Copernicus Climate Data Store...


2019-12-18 15:31:00,615 INFO Request is queued
2019-12-18 15:31:01,775 INFO Request is completed
2019-12-18 15:31:01,777 INFO Downloading http://136.156.132.110/cache-compute-0001/cache/data0/adaptor.mars.internal-1576697460.6607933-31658-9-e6bbea90-0e9e-4640-bad9-829754699a64.grib to /home/matt_s/kadlu/storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2015-03-09_19h.grb2 (312.1K)
2019-12-18 15:31:02,678 INFO Download rate 346.6K/s


processed and inserted 143420 rows. 116500 null values removed, 0 duplicate rows ignored



In [11]:
wavesource2.fetch_windwaveheight(south=south, north=north, 
                                 west=west, east=east, 
                                 start=start, end=end)

/home/matt_s/kadlu/kadlu/geospatial/data_sources/wwiii.py:80: UserWarning: resolution selection not implemented yet. defaulting to 0.5deg resolution
  warnings.warn("resolution selection not implemented yet. defaulting to 0.5deg resolution")



downloading multi_1.glo_30m.hs.201503.grb2 from NOAA WaveWatch III...
preparing multi_1.glo_30m.hs.201503.grb2 for the database...
	processing msg 72 of 249 monthly messages for 2015-03-09 21:00:00
processed and inserted 73386 rows. 163558 null values removed, 210896 duplicate rows ignored


## Loading (example)

In [ ]:
temp, tlat, tlon, ttime, tdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

salinity, slat, slon, stime, sdep = \
temp_salinity_source.load_temp(south=south, north=north, west=west, east=east, 
                               start=start, end=end, top=top, bottom=bottom)

In [ ]:
bathy, blat, blon = bathysource.load_bathymetry(south=south, north=north, west=west, east=east)

In [12]:
wave, wlat, wlon, wtime = \
wavesource1.load_windwaveswellheight(south=south, north=north, 
                                     west=west, east=east, 
                                     start=start, end=end)

In [13]:
wave, wlat, wlon, wtime = \
wavesource2.load_windwaveheight(south=south, north=north, 
                                west=west, east=east, 
                                start=start, end=end)

## SoundSpeed -- loading data

In [14]:
from kadlu.geospatial.ocean import Ocean
from kadlu.sound.sound_speed import SoundSpeed 
from kadlu.geospatial.data_sources.fetch_util import storage_cfg

In [19]:
step = 1 # 1 deg -> approx. 100km at equator 

# step through each 1x1 degree box in selected area 
for ymin in np.arange(south, north, step):
    for xmin in np.arange(west, east, step):
        xmax = xmin + step
        ymax = ymin + step
        
        # load bathymetry
        try:
            bathy, blat, blon = bathysource.load_bathymetry(south=ymin, north=ymax, west=xmin, east=xmax)
        except:
            print(f"no bathy data for {(ymin, xmin)}, skipping")
            continue
        
        
        # load salinity and temp from hycom
        try:
            temp, tlat, tlon, ttime, tdep = \
            temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                           start=start, end=end, top=top, bottom=bottom)

            salinity, slat, slon, stime, sdep = \
            temp_salinity_source.load_temp(south=ymin, north=ymax, west=xmin, east=xmax, 
                                           start=start, end=end, top=top, bottom=bottom)
        except:
            print(f"no salinity/temp data for {(ymin, xmin)}, skipping")
            continue
        
        try:
            wave, wlat, wlon, wtime = \
            wavesource2.load_windwaveheight(south=ymin, north=ymax, 
                                                 west=xmin, east=xmax, 
                                                 start=start, end=end)
        except Exception as e :
            print(f"no wave data for {(ymin, xmin)}", end='\t')
            print(e)
            continue

        """
        # instead, just using 5 as a constant wave height for now
        wave = np.array([5 for x in temp])
        """
        oceantest = Ocean(salinity=salinity, temp=temp, bathy=bathy, wave=wave, wave_var='comb. wind wave swell height')
        oceantest.set_origin(lat_ref=tlat, lon_ref=tlon)
        
        sndspd = SoundSpeed(oceantest)._sound_speed(tlat, tlon, tdep, temp, salinity)
        
        print(sndspd, ymin, xmin)


/home/matt_s/kadlu/kadlu/geospatial/data_sources/chs.py:158: UserWarning: no data found for query range, returning empty arrays
  warnings.warn("no data found for query range, returning empty arrays")


no salinity/temp data for (44.013, -70.429), skipping
no wave data for (44.013, -69.429)	no data found, try adjusting query bounds or fetching some


TypeError: loop of ufunc does not support argument 0 of type float which has no callable cos method

In [6]:
from kadlu.geospatial import ocean
from kadlu.sound import sound_speed
from kadlu.geospatial.data_sources.fetch_util import storage_cfg


<module 'kadlu.geospatial.ocean' from '/home/matt_s/kadlu/kadlu/geospatial/ocean.py'>

In [30]:
reload(ocean)

south_tile, west_tile = 48.2, -62.8 # gulf st lawrence
north_tile, east_tile = south_tile +2, west_tile +2
testocean = ocean.Ocean(bathy='CHS', temp='HYCOM', salinity='HYCOM', wave='WWIII').load\
(south=south_tile, north=north_tile, west=west_tile, east=east_tile, 
 top=top, bottom=bottom, start=start, end=end)


In [31]:
#testocean.set_origin(lat_ref=tlat, lon_ref=tlon)

sndspd = sound_speed.SoundSpeed(testocean)._sound_speed(tlat, tlon, tdep, temp, salinity)

AttributeError: 'NoneType' object has no attribute 'set_origin'

In [21]:
tlat * (np.pi/180)

array([0.7686430185572692, 0.7693411662370335, 0.7700392473377705,
       0.7707373950175348, 0.7714355426972991, 0.7721336903770634,
       0.7686430185572692, 0.7693411662370335, 0.7700392473377705,
       0.7707373950175348, 0.7714355426972991, 0.7686430185572692,
       0.7693411662370335, 0.7700392473377705, 0.7707373950175348,
       0.7714355426972991, 0.7721336903770634, 0.7686430185572692,
       0.7693411662370335, 0.7700392473377705, 0.7707373950175348,
       0.7714355426972991, 0.7721336903770634, 0.7728317714778004,
       0.7735299191575646, 0.7686430185572692, 0.7693411662370335,
       0.7700392473377705, 0.7707373950175348, 0.7714355426972991,
       0.7721336903770634, 0.7728317714778004, 0.7735299191575646,
       0.7686430185572692, 0.7693411662370335, 0.7700392473377705,
       0.7707373950175348, 0.7714355426972991, 0.7721336903770634,
       0.7728317714778004, 0.7735299191575646, 0.7742280668373289,
       0.7686430185572692, 0.7693411662370335, 0.7700392473377